In [2]:
# %%
import torch
import numpy as np
import math
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import time
import sys
import os 
from scipy.sparse import linalg
from pathlib import Path
if torch.cuda.is_available():  
    device = "cuda" 
else:  
    device = "cpu" 

torch.set_default_dtype(torch.float64)
pi = torch.tensor(np.pi,dtype=torch.float64)
ZERO = torch.tensor([0.]).to(device)


class model(nn.Module):
    """ ReLU k shallow neural network
    Parameters: 
    input size: input dimension
    hidden_size1 : number of hidden layers 
    num_classes: output classes 
    k: degree of relu functions
    """
    def __init__(self, input_size, hidden_size1, num_classes,k = 1):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, num_classes,bias = False)
        self.k = k 
    def forward(self, x):
        u1 = self.fc2(F.relu(self.fc1(x))**self.k)
        return u1
    def evaluate_derivative(self, x, i):
        if self.k == 1:
            ## ZERO = torch.tensor([0.]).to(device)
            u1 = self.fc2(torch.heaviside(self.fc1(x),ZERO) * self.fc1.weight.t()[i-1:i,:] )
        else:
            u1 = self.fc2(self.k*F.relu(self.fc1(x))**(self.k-1) *self.fc1.weight.t()[i-1:i,:] )  
        return u1
    
    
def show_convergence_order2(err_l2,err_h10,exponent,dict_size,k,d, filename,write2file = False):
    
    if write2file:
        file_mode = "a" if os.path.exists(filename) else "w"
        f_write = open(filename, file_mode)
    
    neuron_nums = [2**j for j in range(2,exponent+1)]
    err_list = [err_l2[i] for i in neuron_nums ]
    err_list2 = [err_h10[i] for i in neuron_nums ] 
    l2_order = -1/2-(2*k + 1)/(2*d)
    h1_order =  -1/2-(2*(k-1)+ 1)/(2*d)
    if write2file:
        f_write.write('dictionary size: {}\n'.format(dict_size))
        f_write.write("neuron num \t\t error \t\t order {:.2f} \t\t h10 error \t\t order {:.2f} \n".format(l2_order,h1_order))
    print("neuron num \t\t error \t\t order {:.2f} \t\t h10 error \t\t order {:.2f} \n".format(l2_order,h1_order))
    for i, item in enumerate(err_list):
        if i == 0: 
            print("{} \t\t {:.6f} \t\t * \t\t {:.6f} \t\t * \n".format(neuron_nums[i],item, err_list2[i] ) )
            if write2file: 
                f_write.write("{} \t\t {} \t\t * \t\t {} \t\t * \n".format(neuron_nums[i],item, err_list2[i] ))
        else: 
            print("{} \t\t {:.6f} \t\t {:.6f} \t\t {:.6f} \t\t {:.6f} \n".format(neuron_nums[i],item,np.log(err_list[i-1]/err_list[i])/np.log(2),err_list2[i] , np.log(err_list2[i-1]/err_list2[i])/np.log(2) ) )
            if write2file: 
                f_write.write("{} \t\t {} \t\t {} \t\t {} \t\t {} \n".format(neuron_nums[i],item,np.log(err_list[i-1]/err_list[i])/np.log(2),err_list2[i] , np.log(err_list2[i-1]/err_list2[i])/np.log(2) ))
    if write2file:     
        f_write.write("\n")
        f_write.close()

def show_convergence_order_latex2(err_l2,err_h10,exponent,k=1,d=1): 
    neuron_nums = [2**j for j in range(2,exponent+1)]
    err_list = [err_l2[i] for i in neuron_nums ]
    err_list2 = [err_h10[i] for i in neuron_nums ] 
    l2_order = -1/2-(2*k + 1)/(2*d)
    h1_order =  -1/2-(2*(k-1)+ 1)/(2*d)
    print("neuron num  & \t $\|u-u_n \|_{{L^2}}$ & \t order $O(n^{{{:.2f}}})$  & \t $ | u -u_n |_{{H^1}}$ & \t order $O(n^{{{:.2f}}})$  \\\ \hline \hline ".format(l2_order,h1_order))
    for i, item in enumerate(err_list):
        if i == 0: 
            print("{} \t\t & {:.6f} &\t\t * & \t\t {:.6f} & \t\t *  \\\ \hline  \n".format(neuron_nums[i],item, err_list2[i] ) )   
        else: 
            print("{} \t\t &  {:.3e} &  \t\t {:.2f} &  \t\t {:.3e} &  \t\t {:.2f} \\\ \hline  \n".format(neuron_nums[i],item,np.log(err_list[i-1]/err_list[i])/np.log(2),err_list2[i] , np.log(err_list2[i-1]/err_list2[i])/np.log(2) ) )


## 1D 


In [6]:

dim = 1
function_name = "gabor1d-m4" 
filename_write = "data/1DOGA-PBE-{}-order.txt".format(function_name)
Nx = 2**10
order = 3
relu_k = 3 
memory = 2**29 

trial_num = 5 
for N in [2**9]: # 2**12,2**14
    errl2_trials = []
    errh10_trials = [] 
    for trial in range(trial_num): 
        print("trial: {}".format(trial))
        exponent = 8 

        num_epochs = 2**exponent  
        plot_freq = num_epochs 

        folder = '/Users/xux0f/Desktop/github/RandomizedOGA/nonlinearPDEs/data-revision1/'
        filename = folder + 'errl2_CGA_1D_{}_neuron_{}_N_{}_randomized_trial_{}.pt'.format(function_name,num_epochs,N,trial)
        errl2 = torch.load(filename)
        filename = folder + 'errh10_CGA_1D_{}_neuron_{}_N_{}_randomized_trial_{}.pt'.format(function_name,num_epochs,N,trial)
        err_h10 = torch.load(filename)
        show_convergence_order_latex2(errl2,err_h10,exponent,k=relu_k,d=dim)
        errl2_trials.append(errl2)
        errh10_trials.append(err_h10)

errl2_trials = torch.stack(errl2_trials, dim=0)
errh10_trials = torch.stack(errh10_trials, dim=0)
errl2_mean = torch.mean(errl2_trials, dim=0)
errh10_mean = torch.mean(errh10_trials, dim=0)
errl2_std = torch.std(errl2_trials, dim=0)
errh10_std = torch.std(errh10_trials, dim=0)

print("average errors")
show_convergence_order_latex2(errl2_mean,errh10_mean,exponent,k=relu_k,d=dim)



trial: 0
neuron num  & 	 $\|u-u_n \|_{L^2}$ & 	 order $O(n^{-4.00})$  & 	 $ | u -u_n |_{H^1}$ & 	 order $O(n^{-3.00})$  \\ \hline \hline 
4 		 & 0.364861 &		 * & 		 9.322015 & 		 *  \\ \hline  

8 		 &  2.965e-01 &  		 0.30 &  		 7.389e+00 &  		 0.34 \\ \hline  

16 		 &  1.338e-02 &  		 4.47 &  		 7.231e-01 &  		 3.35 \\ \hline  

32 		 &  5.698e-04 &  		 4.55 &  		 6.772e-02 &  		 3.42 \\ \hline  

64 		 &  2.122e-05 &  		 4.75 &  		 5.944e-03 &  		 3.51 \\ \hline  

128 		 &  1.013e-06 &  		 4.39 &  		 6.024e-04 &  		 3.30 \\ \hline  

256 		 &  6.662e-08 &  		 3.93 &  		 7.870e-05 &  		 2.94 \\ \hline  

trial: 1
neuron num  & 	 $\|u-u_n \|_{L^2}$ & 	 order $O(n^{-4.00})$  & 	 $ | u -u_n |_{H^1}$ & 	 order $O(n^{-3.00})$  \\ \hline \hline 
4 		 & 0.364856 &		 * & 		 9.322048 & 		 *  \\ \hline  

8 		 &  2.971e-01 &  		 0.30 &  		 7.370e+00 &  		 0.34 \\ \hline  

16 		 &  1.344e-02 &  		 4.47 &  		 7.246e-01 &  		 3.35 \\ \hline  

32 		 &  5.638e-04 &  		 4.58 &  		 6.209e-02 &  	

## 2D 

In [10]:

dim = 2
function_name = "gabor2d-m4" 
Nx = 600
order = 3
relu_k = 3 
memory = 2**29 
rand_deter = 'rand'
trial_num = 5 

for N in [2**9]: # 2**12,2**14
    errl2_trials = []
    errh10_trials = [] 
    for trial in range(trial_num): 
        print("trial: {}".format(trial))
        exponent = 9 

        num_epochs = 2**exponent  
        plot_freq = num_epochs 

        folder = '/Users/xux0f/Desktop/github/RandomizedOGA/nonlinearPDEs/data-revision1/'
        filename = folder + 'err_OGA_2D_{}_neuron_{}_N_{}_{}_trial_{}.pt'.format(function_name,num_epochs,N,rand_deter,trial)
        errl2 = torch.load(filename)
        filename = folder + 'err_h10_OGA_2D_{}_neuron_{}_N_{}_{}_trial_{}.pt'.format(function_name,num_epochs,N,rand_deter,trial)
        err_h10 = torch.load(filename)
        show_convergence_order_latex2(errl2,err_h10,exponent,k=relu_k,d=dim)
        errl2_trials.append(errl2)
        errh10_trials.append(err_h10)

errl2_trials = torch.stack(errl2_trials, dim=0)
errh10_trials = torch.stack(errh10_trials, dim=0)
errl2_mean = torch.mean(errl2_trials, dim=0)
errh10_mean = torch.mean(errh10_trials, dim=0)
errl2_std = torch.std(errl2_trials, dim=0)
errh10_std = torch.std(errh10_trials, dim=0)

print("average errors")
show_convergence_order_latex2(errl2_mean,errh10_mean,exponent,k=relu_k,d=dim)



trial: 0
neuron num  & 	 $\|u-u_n \|_{L^2}$ & 	 order $O(n^{-2.25})$  & 	 $ | u -u_n |_{H^1}$ & 	 order $O(n^{-1.75})$  \\ \hline \hline 
4 		 & 0.188013 &		 * & 		 4.888192 & 		 *  \\ \hline  

8 		 &  1.882e-01 &  		 -0.00 &  		 4.888e+00 &  		 0.00 \\ \hline  

16 		 &  1.887e-01 &  		 -0.00 &  		 4.859e+00 &  		 0.01 \\ \hline  

32 		 &  1.468e-01 &  		 0.36 &  		 4.003e+00 &  		 0.28 \\ \hline  

64 		 &  4.220e-02 &  		 1.80 &  		 1.478e+00 &  		 1.44 \\ \hline  

128 		 &  6.752e-03 &  		 2.64 &  		 3.735e-01 &  		 1.98 \\ \hline  

256 		 &  7.570e-04 &  		 3.16 &  		 6.821e-02 &  		 2.45 \\ \hline  

512 		 &  1.357e-04 &  		 2.48 &  		 1.783e-02 &  		 1.94 \\ \hline  

trial: 1
neuron num  & 	 $\|u-u_n \|_{L^2}$ & 	 order $O(n^{-2.25})$  & 	 $ | u -u_n |_{H^1}$ & 	 order $O(n^{-1.75})$  \\ \hline \hline 
4 		 & 0.188072 &		 * & 		 4.888142 & 		 *  \\ \hline  

8 		 &  1.881e-01 &  		 0.00 &  		 4.887e+00 &  		 0.00 \\ \hline  

16 		 &  1.887e-01 &  		 -0.01 &  		 4.845e+00 

In [ ]:
## std 
neuron_nums = [2**j for j in range(2,exponent+1)]
for i, item in enumerate(neuron_nums):
    print(f'n = {item}, {errh10_std[item]}')

n = 4, 3.614353608664778e-05
n = 8, 0.000238107047206339
n = 16, 0.06542751312108239
n = 32, 0.20082105033372463
n = 64, 0.1246790356590744
n = 128, 0.02443636735204721
n = 256, 0.0034541806113229756
n = 512, 0.0004938161892802627


## 3D

In [14]:

dim = 3 
function_name = "gabor3d-m4" 
Nx = 50
order = 2 
relu_k = 3
memory = 2**29 
exponent =  10 
num_epochs = 2**exponent  
trial_num = 5 
for N_list in [[2**3,2**3,2**3]]: # ,[2**6,2**6],[2**7,2**7] 
    errl2_trials = []
    errh10_trials = []
    for trial in range(trial_num): 
        print("trial: {}".format(trial))
        folder = '/Users/xux0f/Desktop/github/RandomizedOGA/nonlinearPDEs/data-revision1/'
        filename = folder + 'errl2_OGA_3D_{}_relu_{}_neuron_{}_N_{}_randomized_trial_{}.pt'.format(function_name,relu_k,num_epochs,N,trial)
        errl2 = torch.load(filename)
        filename = folder + 'errh10_OGA_3D_{}_relu_{}_neuron_{}_N_{}_randomized_trial_{}.pt'.format(function_name,relu_k,num_epochs,N,trial)
        err_h10 = torch.load(filename)
        show_convergence_order_latex2(errl2,err_h10,exponent,k =relu_k,d = dim)
        errl2_trials.append(errl2)
        errh10_trials.append(err_h10)
    errl2_trials = torch.stack(errl2_trials, dim=0)
    errh10_trials = torch.stack(errh10_trials, dim=0)
    errl2_mean = torch.mean(errl2_trials, dim=0)
    errh10_mean = torch.mean(errh10_trials, dim=0)
    errl2_std = torch.std(errl2_trials, dim=0)
    errh10_std = torch.std(errh10_trials, dim=0)    
    print("average errors")
    show_convergence_order_latex2(errl2_mean,errh10_mean,exponent,k =relu_k,d = dim)


trial: 0
neuron num  & 	 $\|u-u_n \|_{L^2}$ & 	 order $O(n^{-1.67})$  & 	 $ | u -u_n |_{H^1}$ & 	 order $O(n^{-1.33})$  \\ \hline \hline 
4 		 & 0.096945 &		 * & 		 2.561570 & 		 *  \\ \hline  

8 		 &  9.701e-02 &  		 -0.00 &  		 2.561e+00 &  		 0.00 \\ \hline  

16 		 &  9.702e-02 &  		 -0.00 &  		 2.561e+00 &  		 0.00 \\ \hline  

32 		 &  9.694e-02 &  		 0.00 &  		 2.553e+00 &  		 0.00 \\ \hline  

64 		 &  9.054e-02 &  		 0.10 &  		 2.341e+00 &  		 0.12 \\ \hline  

128 		 &  6.943e-02 &  		 0.38 &  		 1.927e+00 &  		 0.28 \\ \hline  

256 		 &  2.798e-02 &  		 1.31 &  		 9.248e-01 &  		 1.06 \\ \hline  

512 		 &  7.433e-03 &  		 1.91 &  		 3.274e-01 &  		 1.50 \\ \hline  

1024 		 &  1.712e-03 &  		 2.12 &  		 1.054e-01 &  		 1.63 \\ \hline  

trial: 1
neuron num  & 	 $\|u-u_n \|_{L^2}$ & 	 order $O(n^{-1.67})$  & 	 $ | u -u_n |_{H^1}$ & 	 order $O(n^{-1.33})$  \\ \hline \hline 
4 		 & 0.096937 &		 * & 		 2.561585 & 		 *  \\ \hline  

8 		 &  9.695e-02 &  		 -0.00 &  		 2.562e+0